**Troubleshooting Scripts - Database Information and Configuration**

Dmitri V. Korotkevitch (MCM, MVP)

email: [dk@aboutsqlserver.com](mailto:dk@aboutsqlserver.com)      blog: [https://aboutsqlserver.com](https://aboutsqlserver.com/) code: [https://github.com/aboutsqlserver/code](https://github.com/aboutsqlserver/code)

SQL Server Advanced Troubleshooting and Performance Tuning (O'Reilly, 2022)      ISBN: 978-1098101923

**Database Configuration and Space Usage** (SQL Server 2016 SP2+)

Pay attention to VLF configuration

In [ ]:
SELECT 
	db.name
	,SUSER_SNAME(db.owner_sid) AS [db_owner]
	,CONVERT(DECIMAL(16,3), ds.cntr_value / 1024.) AS total_database_size_mb
	,CONVERT(DECIMAL(16,3), ls.cntr_value / 1024.) AS log_size_mb
	,CONVERT(DECIMAL(16,3), lu.cntr_value / 1024) AS log_used_mb
	,CONVERT(DECIMAL(5,2), 1. * lu.cntr_value / ls.cntr_value * 100.) AS log_used_percent
	,li.*
	,DATABASEPROPERTYEX (db.name, 'LastGoodCheckDbTime') AS last_checkdb_date
	,fb.backup_finish_date as last_full_backup_date
	,ib.backup_finish_date as last_incremental_backup_date
	,lb.backup_finish_date as last_log_backup_date
	,db.*
FROM 
	sys.databases db WITH (NOLOCK)
		JOIN sys.dm_os_performance_counters AS ds WITH (NOLOCK) ON 
			db.name = ds.instance_name AND 
			ds.counter_name LIKE N'Data File(s) Size (KB)%'
		LEFT OUTER JOIN sys.dm_os_performance_counters ls WITH (NOLOCK) ON 
			db.name = ls.instance_name AND
			ls.counter_name LIKE N'Log File(s) Size (KB)%' AND
			ls.cntr_value > 0 
		LEFT OUTER JOIN sys.dm_os_performance_counters lu WITH (NOLOCK)ON 
			db.name = lu.instance_name AND
			lu.counter_name LIKE N'Log File(s) Used Size (KB)%'
		CROSS APPLY
		(
			SELECT 
				COUNT(*) AS vlf_count
				,MAX(vlf_size_mb) AS max_vlf_size_mb
				,MIN(vlf_size_mb) AS min_vlf_size_mb
				,AVG(vlf_size_mb) AS avg_vlf_size_mb
			FROM sys.dm_db_log_info(db.database_id) li
		) li
		OUTER APPLY
		(
			SELECT bs.backup_finish_date
			FROM msdb.dbo.backupset bs WITH (NOLOCK)
			WHERE bs.[database_name] = db.name AND bs.[type] = 'D'
		) fb
		OUTER APPLY
		(
			SELECT bs.backup_finish_date
			FROM msdb.dbo.backupset bs WITH (NOLOCK)
			WHERE bs.[database_name] = db.name AND bs.[type] = 'I'
		) ib
		OUTER APPLY
		(
			SELECT bs.backup_finish_date
			FROM msdb.dbo.backupset bs WITH (NOLOCK)
			WHERE bs.[database_name] = db.name AND bs.[type] = 'L'
		) lb
ORDER BY 
	db.[database_id] 
OPTION (MAXDOP 1, RECOMPILE);

**Database Configuration and Space Usage** (prior SQL Server 2016 SP2+)

In [ ]:
SELECT 
	SUSER_SNAME(db.owner_sid) AS [db_owner]
	,CONVERT(DECIMAL(16,3), ds.cntr_value / 1024.) AS total_database_size_mb
	,CONVERT(DECIMAL(16,3), ls.cntr_value / 1024.) AS log_size_mb
	,CONVERT(DECIMAL(16,3), lu.cntr_value / 1024) AS log_used_mb
	,CONVERT(DECIMAL(5,2), 1. * lu.cntr_value / ls.cntr_value * 100.) AS log_used_percent
	,db.*
FROM 
	sys.databases db WITH (NOLOCK)
		JOIN sys.dm_os_performance_counters AS ds WITH (NOLOCK) ON 
			db.name = ds.instance_name AND 
			ds.counter_name LIKE N'Data File(s) Size (KB)%'
		LEFT OUTER JOIN sys.dm_os_performance_counters ls WITH (NOLOCK) ON 
			db.name = ls.instance_name AND
			ls.counter_name LIKE N'Log File(s) Size (KB)%' AND
			ls.cntr_value > 0 
		LEFT OUTER JOIN sys.dm_os_performance_counters lu WITH (NOLOCK)ON 
			db.name = lu.instance_name AND
			lu.counter_name LIKE N'Log File(s) Used Size (KB)%'
ORDER BY 
	db.[database_id] 
OPTION (MAXDOP 1, RECOMPILE);

**Database Files**

Pay attention to:

1\. Uneven file sizes in the same filegroup

2\. Auto-growth in percent, especially in LOG files

3\. Non-balanced auto-growth parameters across the  files in the same filegroup

In [ ]:
IF OBJECT_ID('tempdb..#Info') IS NOT NULL
    DROP TABLE #Info;
GO

CREATE TABLE #Info
(
	database_id SMALLINT NULL,
	[database] NVARCHAR(128) NULL,
	file_id INT NOT NULL,
	file_name SYSNAME NOT NULL,
	[filegroup] SYSNAME NULL,
	type_desc NVARCHAR(60) NULL,
	state_desc NVARCHAR(60) NULL,
	size_mb INT NULL,
	space_used_mb INT NULL,
	space_used_percent DECIMAL(5, 3) NULL,
	max_size_mb INT NULL,
	is_percent_growth BIT NOT NULL,
	growth_percent INT NULL,
	growth_mb INT NULL,
	physical_name NVARCHAR(260) NULL
);

EXEC master..sp_MSforeachdb 
N'USE[?];
INSERT INTO #Info(database_id,[database],file_id,file_name,[filegroup],type_desc,state_desc
	,size_mb,space_used_mb,space_used_percent,max_size_mb,is_percent_growth,growth_percent,growth_mb,physical_name)
SELECT 
	DB_ID(''?'') AS [database_id]
	,DB_NAME() AS [database]
    ,df.file_id
	,df.[name] as file_name
	,fg.name as filegroup
	,df.type_desc
	,df.state_desc
	,CONVERT(INT, df.size / 128.0) AS size_mb
	,CONVERT(INT,FILEPROPERTY(df.name, ''SpaceUsed'') / 128.) as space_used_mb
	,CONVERT(DECIMAL(5,3),100. * CONVERT(INT,FILEPROPERTY(df.name, ''SpaceUsed'')) / df.size) as space_used_percent
	,IIF(max_size = -1,NULL,CONVERT(INT,df.max_size / 128.)) AS max_size_mb
	,df.is_percent_growth
	,IIF(df.is_percent_growth = 1,df.growth,NULL) AS growth_percent
	,IIF(df.is_percent_growth = 1,NULL,CONVERT(INT,df.growth / 128.)) AS growth_mb
	,df.physical_name	
FROM 
	sys.database_files df WITH (NOLOCK)
		LEFT JOIN sys.filegroups fg WITH (NOLOCK) ON
			df.data_space_id = fg.data_space_id';

SELECT * 
FROM #Info
ORDER BY [database_id], type_desc, [filegroup], file_id
OPTION (MAXDOP 1,RECOMPILE);

**Database Scoped Configuration**

In [ ]:
IF OBJECT_ID('tempdb..#DSC') IS NOT NULL
    DROP TABLE #DSC;
GO

CREATE TABLE #DSC
(
	database_id SMALLINT NULL
	,[database] NVARCHAR(128) NULL
	,configuration_id INT NULL
	,name NVARCHAR(60) NULL
	,value SQL_VARIANT NULL
	,value_for_secondary SQL_VARIANT NULL
	,is_value_default BIT NULL
);

DECLARE
	@SQL NVARCHAR(MAX) = 
N'USE[?];
INSERT INTO #DSC(database_id,[database],configuration_id,name,value,value_for_secondary,is_value_default)
SELECT 
	DB_ID(''?'') AS [database_id]
	,DB_NAME() AS [database]
    ,configuration_id
	,name
	,value
	,value_for_secondary
	,' + IIF(LEFT(CONVERT(VARCHAR(32),SERVERPROPERTY('ProductVersion')),2) = '13','NULL','is_value_default') + N'
FROM 
	sys.database_scoped_configurations;'

EXEC master..sp_MSforeachdb @SQL;

SELECT * 
FROM #DSC
WHERE database_id >= 5 and database_id <> 32767
ORDER BY configuration_id, [database]
OPTION (MAXDOP 1,RECOMPILE);